In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

In [ ]:
# Initalisation du driver
service = Service('msedgedriver.exe')
driver = webdriver.Edge(service=service)

### Fonctions utilisées pour la page d'accueil

In [ ]:
def Find_date(driver) :
    """Cette fonction permet de rechercher et sélectionner la date du 1er mai
    dans le calendrier du site tout en évitant les erreur du type ElementNotInteractable et ElementNotFound"""
    
    find = False
    # Tant que le mois de mai n'est pas trouvé dans le calendrier, on continue à chercher
    while find == False :
        # Extraire le code source html et vérifier si le mois de mai est présent
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        elements = soup.find_all(class_='ui-datepicker-month')
        for element in elements :
            if 'Mai' in element.get_text():
                find = True # cela signifie qu'on a trouvé le mois de mai on pourra donc sortir d
        if find != True:
            # On essaie de cliquer sur la flèche pour changer de mois
            try :
                arrow_element = driver.find_element(By.CLASS_NAME,'ui-datepicker-next.ui-corner-all')
                arrow_element.click()
            except :
                print("Unable to change the month on the calendar")
                # Comme on a pas pu changer de mois, on essaie de cliquer à nouveau sur le calendrier
                try :
                    calendar = driver.find_element(By.ID,'biglietti_data_pVISIBLE')
                    calendar.click()
                except:
                    print("Calendar not found")
                    
                
    # Choisir le 1er mai
    driver.find_element(By.XPATH,'//*[@id="ui-datepicker-div"]/div[2]/table/tbody/tr[1]/td[3]/a').click()

In [ ]:
def navigate_in_homepage(driver,depart,arrivee):
    """ Cette fonction permet de naviguer sur la page d'accueil de trenitalia en indiquant la date sur le calendrier, la gare
    de départ et d'arrivée, l'heure et enfin de cliquer sur rechercher pour aller vers la page suivante des horaires"""
    
    web_link= "https://www.trenitalia.com/trenitalia-france.html"
    driver.get(web_link)
    time.sleep(3)

    # Désactivation des cookies
    try : 
        cookies = driver.find_element(By.ID,'onetrust-reject-all-handler')
        cookies.click()
    except:
        print("cookie not found")

    # Sélectionner la date 1er mai à partir du calendrier
    Find_date(driver)

    # Trouvez les champs de la gare de départ et de la gare d'arrivée
    depart = driver.find_element(by='id',value ='biglietti_fromNew')
    arrivee = driver.find_element(by='id',value ='biglietti_toNew')

    # Ecrire le nom des gares de départ et d'arrivée
    depart.send_keys(departure)
    arrivee.send_keys(arrival)

    # Sélectionner le bouton heure
    driver.find_element(By.XPATH,'//*[@id="biglietti_ora_p_but"]').click()
    time.sleep(0.5)
    # Sélectionner l'heure minuit
    driver.find_element(By.XPATH,'//*[@id="1p"]').click()
    # Cliquer sur le bouton rechercher pour aller sur la page des horaires du trajet 
    button = driver.find_element(By.XPATH,'//*[@id="sub-and-carnet"]/button')
    button.click()

### Fonctions utilisées pour la page des horaires de trains

In [ ]:
def get_date_month(driver):
    """Cette fonction permet de récupérer le mois et la date sélectionné (curseur positionné) sur la page des horaires
    Elle permet d'éviter l'erreur AttributeError lors du scraping """
    
    date_selected = None
    mois_selected = None
    
    # Tant que le mois et la date n'ont pas été trouvé on continue d'essayer de le trouver avec beautifoulSoup
    while(date_selected is None and mois_selected is None) :
        # Récupération du code html de la page web
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        
        try :
            # On essaie de récupérer le jour et le mois
            date_selected = soup.find('div',{'class':'card card-calendar au-target'}) \
                .find('li',{'class':'au-target nav-item today cursor-pointer'}) \
                .find('div',{"class": "align-items-center h-100 d-none d-sm-flex"}) \
                .find('strong',{'class':'au-target day-number'}).get_text(strip=True)

            # Récupération du mois sélectionné
            mois_selected = soup.find('div',{'class':'card card-calendar au-target'}) \
                .find('li',{'class':'au-target nav-item today cursor-pointer'}) \
                .find('div',{"class": "align-items-center h-100 d-none d-sm-flex"}) \
                .find('strong',{'class':'au-target month'}).get_text(strip=True)
        except:
            # L'except permet d'éviter l'erreur Attribute error rencontrée à de multiples reprises qui bloque le scraping.
            # Cette erreur est dûe au fait que la page web contenant les horaires mets du temps à s'afficher 
            # ce qui explique l'utilisation de la boucle while
            print("Date and month not found")
            
    return date_selected,mois_selected

In [ ]:
def scroll_down(driver):
    """Cette fonction permet de scroller complètement vers le bas pour récupérer l'ensemble des horaires de train 
    sur la même page web"""
    
    scroll =True
    # On scrolle vers le bas tant qu'il reste encore d'autres horaires sur la page web
    while scroll ==True:
        # On scrolle complètement vers le bas
        height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
        # Récupération du code html de la page web
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        # On recherche l'élement el qui indique si d'autres horaires de trains sont présentes sur la page web
        el = soup.find('div',attrs = {'class':'col-12 text-center mt-2 au-target'})
        if el==None :
            scroll = False

In [ ]:
def change_date(driver) :
    """ Cette fonction permet de changer de date sur la page web des horaires d'un même trajet """
    
    # On revient tout en haut de la page web
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    date_changed = False
    # On essaie de changer la date tant que cela ne fonctionne pas
    while date_changed == False :
        if date_changed != True:
            next_date = driver.find_element(By.ID,'forwardDays')
            try :
                next_date.click()
                date_changed = True
            except :
                print("Date not changed")
                # Ables to avoid element not interactable
                date_changed = False

### Scraping de l'ensemble des trajets de façon automatique
Ce code utilise l'ensemble des fonctions présentées ci-dessus pour scraper le site internet trenitalia sans générer d'erreurs et de façon automatique 

In [ ]:
# Généralisation pour l'automatiser sur l'intégralité des stations

# Initialisation des listes contenants chacune une colonne du dataframe contenant l'ensemble des données
prix = []
station_departure = []
station_arrival = []
traject_lenght = []
date_departure = []
hour_departure = []
hour_arrival = []

# Noms des trajets à scraper (listes de tuples de stations)
routes = [
    ("Venezia ( Tutte Le Stazioni )", "Napoli ( Tutte Le Stazioni )"),
    ("Venezia ( Tutte Le Stazioni )", "Bari ( Tutte Le Stazioni )"),
    ("Milano ( Tutte Le Stazioni )", "Bari ( Tutte Le Stazioni )"),
    ("Milano ( Tutte Le Stazioni )", "Napoli ( Tutte Le Stazioni )"),
    ("Milano ( Tutte Le Stazioni )", "Pescara Centrale"),
    ("Roma ( Tutte Le Stazioni )", "Bari ( Tutte Le Stazioni )"),
    ("Roma ( Tutte Le Stazioni )", "Palermo ( Tutte Le Stazioni )"),
    ("Napoli ( Tutte Le Stazioni )", "Genova ( Tutte Le Stazioni )"),
    ("Napoli ( Tutte Le Stazioni )", "Palermo ( Tutte Le Stazioni )"),
    ("Napoli ( Tutte Le Stazioni )", "Torino ( Tutte Le Stazioni )"),
    ("Torino ( Tutte Le Stazioni )", "Bari ( Tutte Le Stazioni )")
]

# Pour chaque trajet
for departure, arrival in routes:
    #On navige dans la page d'accueil vers les horaires
    navigate_in_homepage(driver,departure,arrival)
    time.sleep(3)
    stop_scrap = False
    
    # On scrappe l'intégralité du mois de mai (la variable stop_scrap permet de s'en assurer)
    while stop_scrap == False :
        # On récupère la date et le mois ou le curseur est positionné sur la page des horaires
        date_selected, mois_selected = get_date_month(driver)
        
        # Arrêt du scraping pour la route concernée si on est plus en mai on va sortir de la boucle while et changer de trajet
        if mois_selected !='MAI' :
            stop_scrap = True

        else :
            # On scrolle complètement vers le bas la page web
            scroll_down(driver)
            time.sleep(3)
            # Récupération du code html
            html = driver.page_source
            soup = BeautifulSoup(html,"html.parser")
            # Récupération du div contenant l'ensemble des trains
            div = soup.find('div', {'id': 'solution-accordion','class':'w-100'})
            # récupération des trains de la page
            trains = div.find_all('div',attrs = {'class':'row au-target','role':'region'})
            # Pour chaque train
            for train in trains :
                try :
                    # Si le trajet n'a pas de correspondances on récupère les données 
                    if 'correspondance' not in train.find('b',{'class':'au-target'}).get_text(strip=True):
                        # Prix
                        prix.append(train.find('title2',{'class' : 'black text-primary ml-1 mb-0'}).get_text(strip=True))
                        # Gare de départ
                        station_departure.append(train.find('div',{'class' : 'col-4 text-nowrap'}).get_text(strip=True))
                        # Gare d'arrivée
                        station_arrival.append(train.find('div',{'class' : 'au-target col-3 offset-5 pl-0 text-nowrap'}).get_text(strip=True))
                        # Durée du trajet
                        traject_lenght.append(train.find('caption1',{'class' : 'text-dark center mx-3'}).get_text(strip=True))
                        # Heure de départ
                        hour_departure.append(train.find('body2',{'class' : 'text-secondary bold m-0 mr-3'}).get_text(strip=True))
                        # Heure d'arrivée
                        hour_arrival.append(train.find('body2',{'class' : 'text-secondary bold m-0 ml-3'}).get_text(strip=True))
                        # Date de départ
                        date_departure.append(datetime(year=2024, month=5, day=int(date_selected)))
                except :
                    # Cas ou l'on ne trouve pas de trajets sans correspondance
                    print("Not found")
            # Changement de la date sur la page des horaires
            change_date(driver)
            
# Récupération de l'ensemble des données dans un dataframe converti ensuite en csv          
data = {
    'station_departure': station_departure,
    'station_arrival': station_arrival,
    'hour_departure': hour_departure,
    'hour_arrival': hour_arrival,
    'date_departure': date_departure,
    'traject_lenght': traject_lenght,
    'prix': prix,
}

df = pd.DataFrame(data)
df.to_csv("trenitalia_data.csv")
driver.quit()